In [68]:
import pandas as pd
import numpy as np
import requests
import matplotlib as plt
import time

# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 'RGAPI-68bcabc9-5f8d-4032-9378-3d7f707f3203'

# 내 소환사명 체크하기
summonerName = "으른고양이"
requesturl = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/"+summonerName+"?api_key="+api_key
# requesturl = "https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/%EC%9C%BC%EB%A5%B8%EA%B3%A0%EC%96%91%EC%9D%B4?api_key=RGAPI-7e3ee2a5-1074-414e-903f-4dd3ffc749a0"
r = requests.get(requesturl)
# https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/%EC%9C%BC%EB%A5%B8%EA%B3%A0%EC%96%91%EC%9D%B4?

In [35]:
# 불러온 데이터 확인
r.json()

{'id': 'rTSdd9vZrO7_6io1ZAKta98vKq8zggGsMvjMGQ5i65UtZw',
 'accountId': 'FbbYWky4v8UHLZ3BGZf213l4IqGNcVhDGO6A6d3t4Ijk',
 'puuid': 'h9yfRKpJKADHKSDO0aGHDtQza_eY5Gl5z0JQ-oOfAWScasBSCYiaJt2uMpFOFKwaQDA4hTKqp9XA1w',
 'name': '으른고양이',
 'profileIconId': 596,
 'revisionDate': 1628689575445,
 'summonerLevel': 310}

In [40]:
# 불러온 데이터 중 소환사1의 데이터 확인
accountId = r.json()['accountId']
accountId

'FbbYWky4v8UHLZ3BGZf213l4IqGNcVhDGO6A6d3t4Ijk'

In [44]:
match_request = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/"+accountId+"?api_key="+api_key
match_get = requests.get(match_request)
new_games = pd.json_normalize(match_get.json()['matches'])['gameId']

In [70]:
import time

In [76]:
my_games = new_games
start_index = 100
while True:
#     print(type(start_index))
    match_requesturl = "https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/"+accountId+"?beginIndex="+str(start_index)+"&api_key="+api_key
    match_return = requests.get(match_requesturl)
    if len(pd.json_normalize(match_return.json()['matches'])) == 0:
        break
    new_games = pd.json_normalize(match_return.json()['matches'])['gameId']
    my_games = pd.concat([my_games, new_games])
    start_index = start_index+100
    time.sleep(1)

In [78]:
my_df = pd.DataFrame()

In [80]:
for gameid in my_games:
    match_detail_requesturl = "https://kr.api.riotgames.com/lol/match/v4/matches/"+str(gameid)+"?api_key="+api_key
    match_detail_return = requests.get(match_detail_requesturl)
    df_new = pd.json_normalize(match_detail_return.json()['participants'])
    participant_me = pd.json_normalize(match_detail_return.json()['participantIdentities'])
    participant_me = participant_me[participant_me['player.accountId']==r.json()['accountId']]
    df_new = df_new[df_new['participantId']==int(participant_me['participantId'])]
    my_df = my_df.append(df_new[['teamId','stats.win']])
    time.sleep(1)

In [81]:
def count_WIN(x):
    d = []
    d.append(x.teamId.count()),
    d.append(x['stats.win'].sum())
    
    return pd.Series(d, index = ['count','win_count'])

my_df = my_df.groupby('teamId', as_index=False).apply(count_WIN)

In [82]:
my_df['WR'] = my_df['win_count'] / my_df['count']
my_df

,teamId,count,win_count,WR
0,100,1336,726,0.543413
1,200,1417,696,0.491179
